In [ ]:
from dgps import *
from methods import *
import numpy as np
import random
import pandas as pd
import seaborn as sns
import pickle

# Parametric Verma Tests

200 trials for each sample size 200 to 20000. On each trial pick randomly from 4 ADMGs, two that satisfy front-door and two that don't. Use parametric primal and dual and compute true positive/false positive rates.

In [ ]:
# the sample sizes tried
sample_sizes = [200, 500, 1000, 1500, 2000, 5000, 20000]
num_trials = 200 # number of datasets to generate for each sample size
# the models, two of which satisfy front-door and two that don't
models = ["fd_admg1", "fd_admg2", "nonfd_admg1", "nonfd_admg2"]
alpha = 0.05

# set seeds
np.random.seed(42)
random.seed(42)

dual_tprs, dual_fprs, dual_accuracies = [], [], []
primal_tprs, primal_fprs, primal_accuracies = [], [], []
dual_bias_tps = []
dual_bias_fps = []
primal_bias_tps = []
primal_bias_fps = []


# go over each sample size
for num_samples in sample_sizes:
    
    # keep track of true positives, false positives etc.
    dual_tps, dual_fps, dual_tns, dual_fns = 0, 0, 0, 0
    primal_tps, primal_fps, primal_tns, primal_fns = 0, 0, 0, 0

    # try num_trials number of models
    for _ in range(num_trials):
        
        # generate random data 
        model = random.choice(models)
        data = None
        satisfies_fd = None
        if model == "fd_admg1":
            data = fd_admg1(num_samples)
            satisfies_fd = True
        elif model == "fd_admg2":
            data = fd_admg2(num_samples)
            satisfies_fd = True
        elif model == "nonfd_admg1":
            data = nonfd_admg1(num_samples)
            satisfies_fd = False
        elif model == "nonfd_admg2":
            data = nonfd_admg2(num_samples)
            satisfies_fd = False
        else:
            assert(False)
        
        dual_weights_A1 = parametric_dual_weights(data, "M", ["Z", "A"], "A", 1)
        dual_weights_A0 = parametric_dual_weights(data, "M", ["Z", "A"], "A", 0)
        primal_weights = parametric_primal_weights(data, "A", "Y", ["Z"], ["Z", "A", "M"])
        dual_pval = weighted_lr_test(data, "Y", "Z", ["M"], dual_weights_A1)
        primal_pval = weighted_lr_test(data, "Y", "Z", ["M"], primal_weights)
        
        dual_fd = True if dual_pval > alpha else False
        primal_fd = True if primal_pval > alpha else False
        
        if num_samples == 5000:
            dual_ace = dual_ipw(data, "Y", dual_weights_A0, dual_weights_A1)
            primal_ace = primal_ipw(data, "Y", "A", primal_weights)
            backdoor_ace = parametric_backdoor(data, "Y", "A", ["Z", "U1", "U2"])

        if dual_fd and satisfies_fd:
            dual_tps += 1
            if num_samples == 5000:
                dual_bias_tps.append(np.abs(dual_ace - backdoor_ace))
        elif dual_fd and not satisfies_fd:
            dual_fps += 1
            if num_samples == 5000:
                dual_bias_fps.append(np.abs(dual_ace - backdoor_ace))
        elif not dual_fd and satisfies_fd:
            dual_fns += 1
        else:
            dual_tns += 1
            
        if primal_fd and satisfies_fd:
            primal_tps += 1
            if num_samples == 5000:
                primal_bias_tps.append(np.abs(primal_ace - backdoor_ace))
        elif primal_fd and not satisfies_fd:
            primal_fps += 1
            if num_samples == 5000:
                primal_bias_fps.append(np.abs(primal_ace - backdoor_ace))
        elif not dual_fd and satisfies_fd:
            primal_fns += 1
        else:
            primal_tns += 1

    # store and print outputs
    dual_tprs.append(round(dual_tps/(dual_tps+dual_fns), 3))
    dual_fprs.append(round(dual_fps/(dual_fps+dual_tns), 3))
    dual_accuracies.append(round((dual_tps + dual_tns)/num_trials, 3))
    primal_tprs.append(round(primal_tps/(primal_tps+primal_fns), 3))
    primal_fprs.append(round(primal_fps/(primal_fps+primal_tns), 3))
    primal_accuracies.append(round((primal_tps + primal_tns)/num_trials, 3))
    print("Sample size =", num_samples)
    print("Dual TPR, FPR, type I rate, accuracy = ",
          dual_tprs[-1],
          dual_fprs[-1],
          round(dual_fns/(dual_tps+dual_fns), 3), # Type I error rate formula
          dual_accuracies[-1])
    print("Primal TPR, FPR, type I rate, accuracy = ",
          primal_tprs[-1],
          primal_fprs[-1],
          round(primal_fns/(primal_tps+primal_fns), 3), # Type I error rate formula
          primal_accuracies[-1])
    
print("Average bias in dual true positives", np.mean(dual_bias_tps), np.std(dual_bias_tps))
print("Average bias in dual false positives", np.mean(dual_bias_fps), np.std(dual_bias_fps))
print("Average bias in primal true positives", np.mean(primal_bias_tps), np.std(primal_bias_tps))
print("Average bias in primal false positives", np.mean(primal_bias_fps), np.std(primal_bias_fps))

# Estimation

Try estimation with IV, primal, dual. One graph has all IV assumptions and front-door assumptions satisfied. One has only front-door satisfied, and one has some non-linearities in there.

In [ ]:
# first do one where only front-door is valid but IV is not
np.random.seed(100)
random.seed(100)
num_bootstraps = 200
# sample_sizes = [200, 500, 1000, 2000, 3000, 4000, 5000]
sample_sizes = [200, 500, 1000, 2000, 5000]
estimates_df = pd.DataFrame(columns=["samples", "method", "ACE estimate"])
large_df = fd_admg1(200000)
ace_truth = parametric_backdoor(large_df, "Y", "A", ["U1", "U2", "Z"])

for num_samples in sample_sizes:
    
    for _ in range(num_bootstraps):
        
        data = large_df.sample(num_samples, replace=False)
        data.reset_index(drop=True, inplace=True)

        # dual estimation
        dual_weights_A1 = parametric_dual_weights(data, "M", ["Z", "A"], "A", 1)
        dual_weights_A0 = parametric_dual_weights(data, "M", ["Z", "A"], "A", 0)
        ace_dual = dual_ipw(data, "Y", dual_weights_A0, dual_weights_A1)
        estimates_df = estimates_df.append({"samples": num_samples, "method": "Dual IPW", "ACE estimate": ace_dual}, ignore_index=True)

        # primal estimation
        primal_weights = parametric_primal_weights(data, "A", "Y", ["Z"], ["Z", "A", "M"])
        ace_primal = primal_ipw(data, "Y", "A", primal_weights)
        estimates_df = estimates_df.append({"samples": num_samples, "method": "Primal IPW", "ACE estimate": ace_primal}, ignore_index=True)

        # iv estimation
        ace_iv = parametric_iv(data, "Y", "A", "Z")
        estimates_df = estimates_df.append({"samples": num_samples, "method": "IV", "ACE estimate": ace_iv}, ignore_index=True)
    
estimates_df.to_pickle("results/ace_from_noniv_admg.pkl") 

In [ ]:
# now do one where both are valid
np.random.seed(42)
random.seed(42)
num_bootstraps = 200
sample_sizes = [200, 500, 1000, 2000, 5000]
estimates_df = pd.DataFrame(columns=["samples", "method", "ACE estimate"])
large_df = iv_admg1(200000)
ace_truth = parametric_backdoor(large_df, "Y", "A", ["U1", "U2", "Z"])

for num_samples in sample_sizes:
    
    for _ in range(num_bootstraps):
        
        data = large_df.sample(num_samples, replace=False)
        data.reset_index(drop=True, inplace=True)

        # dual estimation
        dual_weights_A1 = parametric_dual_weights(data, "M", ["Z", "A"], "A", 1)
        dual_weights_A0 = parametric_dual_weights(data, "M", ["Z", "A"], "A", 0)
        ace_dual = dual_ipw(data, "Y", dual_weights_A0, dual_weights_A1)
        estimates_df = estimates_df.append({"samples": num_samples, "method": "Dual IPW", "ACE estimate": ace_dual}, ignore_index=True)

        # primal estimation
        primal_weights = parametric_primal_weights(data, "A", "Y", ["Z"], ["Z", "A", "M"])
        ace_primal = primal_ipw(data, "Y", "A", primal_weights)
        estimates_df = estimates_df.append({"samples": num_samples, "method": "Primal IPW", "ACE estimate": ace_primal}, ignore_index=True)

        # iv estimation
        ace_iv = parametric_iv(data, "Y", "A", "Z")
        estimates_df = estimates_df.append({"samples": num_samples, "method": "IV", "ACE estimate": ace_iv}, ignore_index=True)
    
estimates_df.to_pickle("results/ace_from_iv_admg.pkl")

# Machine Learning Verma Tests

In [ ]:
# the sample sizes tried
# NOTE: takes a while to run
sample_sizes = [200, 500, 1000, 1500, 2000, 5000, 10000, 20000]
num_trials = 100 # number of datasets to generate for each sample size
# the models, two of which satisfy front-door and two that don't
models = ["fd_admg1", "nonfd_admg2"]
alpha = 0.05

# set seeds
np.random.seed(42)
random.seed(42)

# go over each sample size
for num_samples in sample_sizes:
    
    # keep track of true positives, false positives etc.
    dual_tps, dual_fps, dual_tns, dual_fns = 0, 0, 0, 0
    primal_tps, primal_fps, primal_tns, primal_fns = 0, 0, 0, 0

    # try num_trials number of models
    for trial in range(num_trials):
        
        if trial%10 == 0:
            print(num_samples, trial)

        # generate random data 
        model = random.choice(models)
        data = None
        satisfies_fd = None
        if model == "fd_admg1":
            data = fd_admg_nonlinear(num_samples)
            satisfies_fd = True
        elif model == "fd_admg2":
            data = fd_admg2(num_samples)
            satisfies_fd = True
        elif model == "nonfd_admg1":
            data = nonfd_admg1(num_samples)
            satisfies_fd = False
        elif model == "nonfd_admg2":
            data = nonfd_admg2(num_samples)
            satisfies_fd = False
        else:
            assert(False)
        
        dual_weights_A1 = ml_dual_weights(data, "M", ["Z", "A"], "A", 1)
        dual_pval = fcit_test(data, "Y", "Z", ["M"], dual_weights_A1)
        
        dual_fd = True if dual_pval > alpha else False
        
        if dual_fd and satisfies_fd:
            dual_tps += 1
        elif dual_fd and not satisfies_fd:
            dual_fps += 1
        elif not dual_fd and satisfies_fd:
            dual_fns += 1
        else:
            dual_tns += 1

    # store and print outputs
    dual_tprs.append(round(dual_tps/(dual_tps+dual_fns), 3))
    dual_fprs.append(round(dual_fps/(dual_fps+dual_tns), 3))
    dual_accuracies.append(round((dual_tps + dual_tns)/num_trials, 3))

    print("Sample size =", num_samples)
    print("Dual TPR, FPR, type I rate, accuracy = ",
          dual_tprs[-1],
          dual_fprs[-1],
          round(dual_fns/(dual_tps+dual_fns), 3), # Type I error rate formula
          dual_accuracies[-1])
    
    
data.to_pickle("results/ml_verma_tests.pkl")